# CrossFuse-XGBoost

CrossFuse-XGBoost is a maximum recommended daily dose (MRDD) predictor for new drugs, especially for the drug in the first-in-human (FIH) trial. It achieves a high prediciton accuracy by using technqiues like multifeature fusion, cross-validation screening and extreme gradient boosting.

To leverage ___CrossFuse-XGBoost___, one can refer to the following sample program.

 ## 1. Configure environment 

In [ ]:
import pandas as pd
import numpy as np
import time
from xgboost import XGBRegressor,XGBClassifier
import ml_methods as mm
import importlib as imp
imp.reload(mm)

In [2]:
# Basic variable definition
data_path = '../data/'

## 2. Feature assembly

### • Load data

In [4]:
con_df_path = '{}/training_set.pkl'.format(data_path)
con_df = pd.read_pickle(con_df_path)

 ### • Construct feature vectors (X) using the appropriate feature combination.

In [5]:
X_all=con_df.iloc[:,7:32905]

In [6]:
"""data splitting"""
# Features included in the data：admet2_list, stitch_list, targets_list, circul_list, maccsk_list, pubchem_list, 
#                              rdkit_list,md_list, avail_cal_list, cmax_ld50_list,dose_cal_list,e3fp_list,ctd_list

# ADMET2 data, [:,7:95], 88 columns
admet2_list=list(range(0,88))
X_admet2=X_all.iloc[:,admet2_list]

# STITCH data, [:, 88:17001], 16913 columns
stitch_list=list(range(88,17001))
X_stitch=X_all.iloc[:,stitch_list]

# DrugBank data, [:, 17001:18534], 1533 columns
targets_list=list(range(17001,18534))
X_targets=X_all.iloc[:,targets_list]

# CircularFingerprint, [:, 18534:19134 ], 600 columns
circul_list=list(range(18534,19134))
X_circul=X_all.iloc[:,circul_list]

# MACCSKeysFingerprint，[:, 19134:19301 ], 167 columns
maccsk_list=list(range(19134,19301))
X_maccsk=X_all.iloc[:,maccsk_list]

# PubChemFingerprint，[:, 19301:20182 ], 881 columns
pubchem_list=list(range(19301,20182))
X_pubchem=X_all.iloc[:,pubchem_list]

# RDKitDescriptors，[:, 20182:20390 ], 208 columns
rdkit_list=list(range(20182,20390))
X_rdkit=X_all.iloc[:,rdkit_list]

# MordredDescriptors，[:, 20390:22003 ], 1613 columns
md_list=list(range(20390,22003))
X_md=X_all.iloc[:,md_list]

# Bioavailability data, [:, 22003:22005 ], 2 columns
avail_cal_list=list(range(22003,22005))
X_avail=X_all.iloc[:,avail_cal_list]

# Cmax and LD50, [:, 22005:22007 ], 2 columns
cmax_ld50_list=list(range(22005,22007))
X_cmax_ld50=X_all.iloc[:,cmax_ld50_list]

# Feature derived data, [:, 22007:22010 ], 3 columns
dose_cal_list=list(range(22007,22010))
X_dose_cal=X_all.iloc[:,dose_cal_list]

# E3fpFingerprint, [:, 22010:26106 ], 4096 columns
e3fp_list=list(range(22010,26106))
X_e3fp=X_all.iloc[:,e3fp_list]

# CTD data, [:, 26106:32898 ], 6792 columns
ctd_list=list(range(26106,32898))
X_ctd=X_all.iloc[:,ctd_list]

In [7]:
# combination of features
col_list = admet2_list + stitch_list + targets_list + circul_list + maccsk_list + pubchem_list + rdkit_list + cmax_ld50_list + dose_cal_list + ctd_list
X_col_list = X_all.iloc[:,col_list]
print(X_col_list.shape)

(1216, 27187)


### •  Construct labels (y)

In [8]:
# reg label
y_regress_path = '{}/y_regress.pkl'.format(data_path)
y_regress = mm.read_pkl(y_regress_path)

# class label
y_class_path = '{}/y_class.pkl'.format(data_path)
y_class = mm.read_pkl(y_class_path)


## 3. Data check

In [9]:
print(X_col_list.shape,y_regress.shape,y_class.shape)

(1216, 27187) (1216,) (1216,)


In [10]:
np.isinf(X_col_list).sum().sum(), X_col_list.isna().sum().sum()

(0, 0)

In [11]:
pd.DataFrame(y_regress).describe(),pd.DataFrame(y_class).value_counts()

(                 0
 count  1216.000000
 mean     -4.746616
 std       2.737119
 min     -18.173422
 25%      -6.404368
 50%      -4.369490
 75%      -2.816797
 max       2.383944,
 0    608
 1    608
 dtype: int64)

In [12]:
 pd.DataFrame(y_regress).isna().sum(), pd.DataFrame(y_class).isna().sum()

(0    0
 dtype: int64,
 0    0
 dtype: int64)

## 4. Model training

In [13]:
params_file = '{}/xgboostr_parameters.pkl'.format(data_path)
xgbr_other_params = mm.read_pkl(params_file)

In [14]:
xgboostr_model = XGBRegressor(**xgbr_other_params)

In [ ]:
train_index_list,test_index_list,Y_test_list,Y_pre_list,df_2list=mm.cal_correlation('XGBoost',xgboostr_model,X_col_list,y_regress )

In [15]:
"""Feature Importance Accumulation for 40-Fold Cross-Validation"""
feature_imp_count_df_path = '{}/feature_imp_count_df.pkl'.format(data_path)
feature_imp_count_df = pd.read_pickle(feature_imp_count_df_path)

In [16]:
feature_imp_count_df[feature_imp_count_df['count']==40].shape

(298, 43)

In [17]:
feature_imp_df = feature_imp_count_df[feature_imp_count_df['importance_sum']>0].sort_values(by='importance_sum',ascending=False)

In [ ]:
# Add features one by one
for i in range(1,298):
    feature_imp_cols = list(feature_imp_df['var'])[:i]
    print(' ===== feature_num:',i,' ==== ')
    train_index_list3,test_index_list3,Y_test_list3,Y_pre_list3,df_2list3=mm.cal_correlation('XGBoost',xgboostr_model,X_col_list.loc[:,feature_imp_cols],y_regress )

### • Calculate pearson correlations among the top 241 features

In [18]:
import pandas as pd
feature_imp_count_df = pd.read_csv('../data/feature_imp_count_df.csv',index_col=0)
feature_imp_count_list = list(feature_imp_count_df.iloc[:,0])
feature241_list = list(feature_imp_count_df.iloc[:241,0])

In [19]:
X_imp_count_df = X_all.loc[:,feature_imp_count_list]
X241_df = X_all.loc[:,feature241_list]

In [20]:
X241_corr = X241_df.corr()
X241_corr

,FDAMDD,cal_Cmax_CL,circual_364,rdkit_125,pubchem_860,maccsk_128,dcmax_pred211115,rdkit_29,rdkit_122,9606.ENSP00000351686,...,rdkit_62,rdkit_121,circual_500,CL,circual_214,rdkit_88,pubchem_698,CYP2C19-sub,NR-AhR,rdkit_17
FDAMDD,1.000000,0.005976,0.135491,-0.073477,0.293821,0.334256,0.011055,0.248357,0.275006,-0.143965,...,-0.159276,0.439687,0.099342,0.209179,0.018811,0.071021,0.237130,0.310145,0.091696,-0.118820
cal_Cmax_CL,0.005976,1.000000,-0.018690,-0.019858,-0.017446,0.011518,0.508063,-0.010138,-0.003145,-0.011737,...,0.001589,0.028049,-0.010738,0.016924,-0.027020,0.012047,-0.045683,0.043937,0.025553,0.010943
circual_364,0.135491,-0.018690,1.000000,0.368215,0.271231,0.233910,-0.014981,0.393703,0.359528,-0.016646,...,-0.127755,-0.058770,0.026002,-0.040398,0.076693,0.043463,0.222479,-0.014696,-0.097533,-0.114575
rdkit_125,-0.073477,-0.019858,0.368215,1.000000,0.140509,0.002015,-0.015738,0.321142,0.282422,0.148934,...,-0.112827,-0.466372,-0.067641,-0.183995,-0.097761,-0.113042,0.036821,-0.162566,-0.156256,-0.022131
pubchem_860,0.293821,-0.017446,0.271231,0.140509,1.000000,0.364770,-0.013985,0.250973,0.228164,-0.064972,...,-0.267873,0.244706,0.007237,0.120970,0.008009,0.102905,0.310914,0.132944,-0.158841,-0.123678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rdkit_88,0.071021,0.012047,0.043463,-0.113042,0.102905,0.191575,-0.020406,0.308225,0.321636,-0.125191,...,0.158267,0.252049,0.235297,0.046866,-0.033869,1.000000,0.055429,-0.004627,-0.011189,-0.030637
pubchem_698,0.237130,-0.045683,0.222479,0.036821,0.310914,0.204344,-0.036503,0.350531,0.353897,-0.124193,...,-0.322185,0.361814,0.088106,0.148575,0.085519,0.055429,1.000000,0.074874,0.145747,-0.050240
CYP2C19-sub,0.310145,0.043937,-0.014696,-0.162566,0.132944,0.248749,0.012474,0.048338,0.065015,-0.160317,...,-0.140175,0.417420,0.119161,0.318360,0.114546,-0.004627,0.074874,1.000000,-0.069591,-0.091462
NR-AhR,0.091696,0.025553,-0.097533,-0.156256,-0.158841,-0.109305,0.063751,0.051852,0.105243,-0.082576,...,0.002306,0.225188,-0.054775,0.058703,0.007000,-0.011189,0.145747,-0.069591,1.000000,0.078594


## 5. SVM comparison

In [ ]:
from sklearn.model_selection import RepeatedKFold,GridSearchCV,RandomizedSearchCV
from xgboost import XGBRegressor #XGBoost
from sklearn.metrics import mean_squared_error,mean_absolute_error
from scipy import stats

# cross validation with pearson correlation.
def cal_correlation(reg_modelname,reg_model,X_all,y_all):
    cv=RepeatedKFold(n_splits=40, n_repeats=1, random_state=12)
    pearsonr_corr_list,mae_list,mse_list=[],[],[]
    for (train_index, test_index) in cv.split(X_all):
        if isinstance(X_all,pd.core.frame.DataFrame):
            X_train=X_all.iloc[train_index,:]
            X_test=X_all.iloc[test_index,:]
        elif isinstance(X_all,np.ndarray):
            X_train=X_all[train_index,:]
            X_test=X_all[test_index,:]
        Y_train=y_all[train_index].ravel()
        Y_test=y_all[test_index].ravel()
        reg_model.fit(X_train,Y_train)
        Y_pre=reg_model.predict(X_test)
        pearsonr_corr=stats.pearsonr(Y_pre,Y_test)[0]
        pearsonr_corr_list.append(pearsonr_corr)
        mae_list.append(mean_absolute_error(Y_pre,Y_test))
        mse_list.append(mean_squared_error(Y_pre,Y_test))
    mm.output_res(pearsonr_corr_list,'')
    return np.mean(pearsonr_corr_list),np.mean(mae_list),np.mean(mse_list)

from sklearn.svm import SVR
svr_ = SVR()
cal_correlation('XGBoost',svr_,X_col_list,y_regress )

## 6. Model tuning

In [21]:
from sklearn.model_selection import RepeatedKFold
def output_res(score_list,algorithm_name):
    score_array=np.array(score_list)
    avg_data=np.around(np.average(score_array),4)
    std_data=np.around(np.std(score_array,ddof=1),4)
    min_data=np.around(min(score_array),4)
    max_data=np.around(max(score_array),4)
    print(algorithm_name,"  avg:",avg_data,"std:",std_data," min:",min_data," max:",max_data)
        
from scipy import stats
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn import metrics
def class_cal_correlation(reg_modelname,reg_model,X_all,y_all):
    pearsonr_corr_list=[]
    spearmanr_corr_list=[]
    a_list = []
    # lists storing the train set and test set for each iteration of cross validation.
    train_index_list,test_index_list,Y_test_list,Y_pre_list,Y_proba_list=[],[],[],[],[] 
    df_2list=[] # stores the feature importance for each iteration of cross validation.
    
    cv=RepeatedKFold(n_splits=40, n_repeats=1, random_state=12)
    for (train_index, test_index) in cv.split(X_all):
        # 
        if isinstance(X_all,pd.core.frame.DataFrame):
            X_train=X_all.iloc[train_index,:]
            X_test=X_all.iloc[test_index,:]
        elif isinstance(X_all,np.ndarray):
            X_train=X_all[train_index,:]
            X_test=X_all[test_index,:]
        Y_train=y_all[train_index].ravel()
        Y_test=y_all[test_index].ravel()
        
        reg_model.fit(X_train,Y_train)
        Y_pre=reg_model.predict(X_test)
        Y_proba = reg_model.predict_proba(X_test)
        
        
        fpr, tpr, thresholds = roc_curve(Y_test,Y_proba[:,1])
        roc_auc = auc(fpr, tpr)
        pearsonr_corr_list.append(roc_auc)
        
    
        # pearsonr_corr=metrics.roc_auc_score(Y_pre,Y_test)
        # pearsonr_corr_list.append(pearsonr_corr)
        
        
        spearmanr_corr=metrics.roc_auc_score(Y_pre,Y_test)
        spearmanr_corr_list.append(spearmanr_corr)
        
        # generates a 2-dimensional list with a shape of (40, n)
        train_index_list.append(train_index.tolist())
        test_index_list.append(test_index.tolist())
        Y_test_list.append(Y_test.tolist())
        Y_pre_list.append(Y_pre.tolist())
        Y_proba_list.append(Y_proba.tolist())
        df_2list.append(reg_model.feature_importances_)
    
    output_res(pearsonr_corr_list,'')
    print(np.round(pearsonr_corr_list,4))
    return train_index_list,test_index_list,Y_test_list,Y_pre_list,Y_proba_list,df_2list

In [22]:
params_file = '{}/xgboostc_parameters.pkl'.format(data_path)
xgbc_other_params = mm.read_pkl(params_file)

In [23]:
xgboostc_model = XGBClassifier(**xgbc_other_params)

In [ ]:
train_index_list,test_index_list,Y_test_list,Y_pre_list,df_2list = class_cal_correlation('XGBoost',xgboostc_model,X_col_list,y_class )

## 7. Independent test for CrossFuse-XGBoost

In [24]:
"""load validation set"""
validata_con_mix_df_path = '{}/external_validation_set.pkl'.format(data_path)
validata_con_mix_df=pd.read_pickle(validata_con_mix_df_path)

In [184]:
X241_df = X_all.loc[:,feature241_list]
xgboostr_model.fit(X241_df,y_regress)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.61,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=1.16, gpu_id=None,
             grow_policy=None, importance_type='gain',
             interaction_constraints=None, learning_rate=0.022, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=0,
             max_depth=7, max_leaves=None, min_child_weight=12, missing=nan,
             monotone_constraints=None, n_estimators=1270, n_jobs=32,
             num_parallel_tree=1, predictor=None, random_state=17, ...)

In [132]:
validata_241df = validata_con_mix_df.loc[:,feature241_list]
y_val_pre=xgboostr_model.predict(validata_241df)
validata_con_mix_df['y_pre'] = y_val_pre

In [67]:
from scipy import stats 
stats.pearsonr(np.log(validata_con_mix_df.iloc[:,5]),validata_con_mix_df['y_pre'])

PearsonRResult(statistic=0.6295470850170055, pvalue=2.872884864935842e-17)